# Curve generation
This notebook can be used to generate "cofactor curves", i.e., curves with a point of given small-order with the rest of the curve having large prime order. Both PARI/gp via cypari2 and [ecgen](https://github.com/J08nY/ecgen) are used and need to be installed.

By default it generates 256 bit curves for all of the divisors from the common "divisor_map"

In [ ]:
import itertools
import random
import time
import json

import cypari2

from subprocess import Popen, PIPE

from tqdm.auto import trange, tqdm

from common import divisor_map

In [ ]:
bits = 256

In [ ]:
pari = cypari2.Pari()
orders = []
for divisor in tqdm(divisor_map["all"]):
    dbits = divisor.bit_length()
    pbits = bits - dbits
    if dbits == bits:
        print(divisor, 1)
        orders.append((divisor, 1, divisor))
        continue
    if dbits > bits:
        print(f"Cannot fill in for divisor with {dbits}.")
        continue
    while True:
        prime = int(pari.randomprime([2**(pbits-1), 2**(pbits+1)]))
        order = divisor * prime
        if order.bit_length() == bits:
            break
    print(divisor, prime)
    orders.append((divisor, prime, order))

In [ ]:
with open("curves_1.json", "r") as f:
    parsed = json.load(f)

In [ ]:
commands = []
processes = []
for divisor, prime, order in orders:
    if str(divisor) in parsed:
        continue
    command = ["./ecgen", "--fp", "-n", f"{divisor},{prime}", "-u", "--points", "none", "-m", "12g", "--threads", "8", str(bits)]
    commands.append(command)
    print(" ".join(command))

random.shuffle(commands)
total = len(commands)

results = {}
errored = {}
running = []
max_procs = 20
with tqdm(total=total, smoothing=0, desc="Computing curves") as pbar:
    while commands or running:
        done = []
        for cmd, start, proc in running:
            now = time.time()
            divisor = int(cmd[3].split(",")[0])
            if (ret := proc.poll()) is not None:
                print(" ".join(cmd))
                err = proc.stderr.read()
                res = proc.stdout.read()
                if err:
                    errored[divisor] = err
                    print(err)
                else:
                    results[divisor] = res
                pbar.update(1)
                done.append((cmd, start, proc))
                if len(results) % 10 == 0:
                    print(f"Results {len(results)}, remaining {len(commands)}, errored {len(errored)}")
            elif now - start > 60 * 60:
                print("Timed out: " + " ".join(cmd))
                errored[divisor] = "timeout"
                proc.kill()
                pbar.update(1)
                done.append((cmd, start, proc))
        for d in done:
            running.remove(d)
        time.sleep(1)
        while len(running) < max_procs and commands:
            command = commands.pop()
            start = time.time()
            proc = Popen(command, stdout=PIPE, stderr=PIPE, text=True)
            running.append((command, start, proc))

In [ ]:
with open("curves_1.json", "r") as f:
    parsed = json.load(f)
if results:
    for div, res in results.items():
        try:
            data = json.loads(res)
            parsed[str(div)] = data[0]
            print("ok", div)
        except:
            print("failed", div)
with open("curves_1.json", "w") as f:
    json.dump(parsed, f)

In [ ]:
len(parsed)